In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [3]:
df = pd.read_csv("lista4.txt", sep=" ")

In [4]:
df.head()

,AGE,EDUC,MEDUC,SIBS,LWAGE,TENURE,URBAN,SOUTH,MARRIED
0,31,12,8.0,1,6.645091,2,1,0,1
1,37,18,14.0,1,6.694562,16,1,0,1
2,33,14,14.0,1,6.715384,9,1,0,1
3,32,12,12.0,4,6.476973,7,1,0,1
4,34,11,6.0,10,6.331502,5,1,0,1


In [5]:
df.isnull().sum()

AGE         0
EDUC        0
MEDUC      78
SIBS        0
LWAGE       0
TENURE      0
URBAN       0
SOUTH       0
MARRIED     0
dtype: int64

In [6]:
family = ["SIBS", "MEDUC"]
indiv = ["EDUC", "AGE", "MARRIED"]
regiao = ["SOUTH", "URBAN"]
emp = ["TENURE"]
categs = {"family": family, "indiv": indiv, "regiao": regiao, "emp": emp}

In [7]:
df = df.dropna().reset_index(drop=True)

In [8]:
X = sm.add_constant(df.drop("LWAGE", axis=1))
y = df[["LWAGE"]]

In [9]:
reg = sm.OLS(y, X).fit()

In [10]:
print(reg.summary())

                            OLS Regression Results                            
Dep. Variable:                  LWAGE   R-squared:                       0.233
Model:                            OLS   Adj. R-squared:                  0.226
Method:                 Least Squares   F-statistic:                     32.25
Date:                Thu, 29 Apr 2021   Prob (F-statistic):           2.08e-44
Time:                        19:17:28   Log-Likelihood:                -353.60
No. Observations:                 857   AIC:                             725.2
Df Residuals:                     848   BIC:                             768.0
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          5.1282      0.169     30.286      0.0

In [11]:
with open("results.html", "w") as file:
    file.write(reg.summary().as_html())

In [12]:
pd.DataFrame(str(reg.summary()).split("\n")).to_csv("results.csv", sep=";", index=False)

In [13]:
for categ in categs:
    index_categ = np.arange(reg.params.size)[reg.params.index.isin(categ)]
    print(reg.f_test(np.identity(len(reg.params))[index_categ, :]))

TypeError: only list-like objects are allowed to be passed to isin(), you passed a [str]

In [ ]:
fam = sm.OLS(y, sm.add_constant(X.drop(categs["family"], axis=1))).fit()

In [ ]:
variables = []
for key in categs.keys():
    if key != "emp":
        for value in categs[key]:
            variables.append(value)

In [ ]:
rest = sm.OLS(y, X[categs["emp"]]).fit()

In [ ]:
print(rest.summary())

In [ ]:
print(fam.summary())

In [ ]:
((fam.ssr - reg.ssr)/(reg.df_model - fam.df_model))/(reg.ssr/(reg.nobs-(reg.df_model+1)))

In [ ]:
def ftests(reg_irres, reg_rest):
    gl_num = (reg_irres.df_model - reg_rest.df_model)
    gl_denom = (reg_irres.nobs-(reg_irres.df_model+1))
    f = ((reg_rest.ssr - reg_irres.ssr)/(reg_irres.df_model - reg_rest.df_model))/(reg_irres.ssr/(reg_irres.nobs-(reg_irres.df_model+1)))

    print(f"GL NUM: {gl_num}")
    print(f"GL DENOM: {gl_denom}")
    print(f"F test: {f}")

In [ ]:
ftests(reg, fam)

In [ ]:
reg_rest = {}

for categ in categs:
    reg_rest[categ] = sm.OLS(y, sm.add_constant(X.drop(categs[categ], axis=1))).fit()

In [ ]:
reg_rest

In [ ]:
for categ in categs:
    ftests(reg, reg_rest[categ])

In [ ]:
reg.nobs

In [ ]:
from scipy.stats import f

In [ ]:
print("valor crítico")
f.ppf(q=1-0.05, dfn=2, dfd=848)

In [ ]:
reg.t_test(reg.params)

In [ ]:
reg.t_test((reg.params.index == "TENURE").astype(int))

In [ ]:
reg